In [23]:
# -*- coding: utf-8 -*-
"""
Created on Sat Apr 18 20:18:15 2020

@author: jeffr
"""

# packages needed
import selenium as sl
from selenium import webdriver
import pandas as pd


In [24]:
# import dataframe of powerhour videos
df = pd.read_csv('./powerhour.csv')

print(df.head(5))

   number        clipid  year          event  \
0       1  HorseRacing1  2015  Triple crown    
1       2         Golf1  2019        Masters   
2       3         Golf7  2017           Open   
3       4         Golf9  2016      Ryder Cup   
4       5        Golf10  2016        Masters   

                                                desc  \
0                                   American Pharaoh   
1  Tiger wins 2019 masters (water balls by compet...   
2                            Spieth Open go get that   
3                   Reed-Rory Ryder cup singles 2016   
4                                    Spieth collapse   

                                                link  duration  
0  https://www.youtube.com/watch?v=WyDcXtVb8_U&t=101        60  
1    https://www.youtube.com/watch?v=pE-aOBGN8UM&t=0        70  
2    https://www.youtube.com/watch?v=-0q3doVGUuE&t=0        80  
3    https://www.youtube.com/watch?v=3fKB0UJxhe4&t=0        68  
4  https://www.youtube.com/watch?v=bMf0JpfpZu8&t=

In [25]:
# create iterables for video loops
itrlink = df.link
itrduration = df.duration

In [27]:
# set selenium browser and load webpage
browser = sl.webdriver.Firefox()
browser.get('https://www.youtube.com')

In [29]:
# video playback

# the intent is:
# use an iterable of the video links to run through each link
# and using a while loop to set a condition of # seconds until the next link is loaded

for i in itrlink:

    t = itrduration
    timeout_start = time.time()

    while time.time() < timeout_start + next(t):   
        browser.get(next(i))

NameError: name 'time' is not defined

SyntaxError: invalid syntax (<ipython-input-21-fa3980bc165b>, line 1)